In [29]:
import pandas as pd

## 1. IMPORTACIÓN DE LOS DATOS

#### Importamos los datos desde el .csv elaborado en el paquete data-preprocessing para estudiar los datos.

In [30]:
path = "../../../data-preprocessing/src/data_preprocessing/data/final_data.csv"
data = pd.read_csv(filepath_or_buffer=path, index_col=0)
data

,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion,hora,distrito,prec
0,1001,2021-12-01,M30,3660,11.0,0,59.0,N,5,00:00:00,10.0,"2,4"
1,1001,2021-12-01,M30,3660,11.0,0,59.0,N,5,00:15:00,10.0,"2,4"
2,1001,2021-12-01,M30,3660,11.0,0,59.0,N,5,00:30:00,10.0,"2,4"
3,1001,2021-12-01,M30,3660,11.0,0,59.0,N,5,00:45:00,10.0,"2,4"
4,1001,2021-12-01,M30,3660,11.0,0,59.0,N,5,01:00:00,10.0,"2,4"
...,...,...,...,...,...,...,...,...,...,...,...,...
51012041,11388,2024-12-31,M30,339,1.0,11,64.0,N,15,22:45:00,5.0,"0,0"
51012042,11388,2024-12-31,M30,288,2.0,10,61.0,N,15,23:00:00,5.0,"0,0"
51012043,11388,2024-12-31,M30,308,2.0,11,60.0,N,15,23:15:00,5.0,"0,0"
51012044,11388,2024-12-31,M30,257,0.0,8,56.0,N,15,23:30:00,5.0,"0,0"


## 2. TRANSFORMACIÓN DE LOS DATOS

#### Estudiamos la presencia de valores nulos

In [51]:
data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 51012046 entries, 0 to 51012045
Data columns (total 12 columns):
 #   Column               Non-Null Count     Dtype  
---  ------               --------------     -----  
 0   id                   51012046 non-null  int64  
 1   fecha                51012046 non-null  object 
 2   tipo_elem            51012046 non-null  object 
 3   intensidad           51012046 non-null  int64  
 4   ocupacion            50965393 non-null  float64
 5   carga                51012046 non-null  int64  
 6   vmed                 50972438 non-null  float64
 7   error                50923283 non-null  object 
 8   periodo_integracion  51012046 non-null  int64  
 9   hora                 51012046 non-null  object 
 10  distrito             51012046 non-null  float64
 11  prec                 51012046 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 4.9+ GB


#### Como se puede observar, las columnas ocupacion, vmed y error presentan valores nulos. Veamos cuántos valores nulos presenta cada columna.

In [52]:
null_values_ocupacion = data['ocupacion'].isnull().sum()
print(f"La columna null_values_ocupacion presenta {null_values_ocupacion} valores nulos")
null_values_vmed = data['vmed'].isnull().sum()
print(f"La columna null_values_vmed presenta {null_values_vmed} valores nulos")
null_values_error = data['error'].isnull().sum()
print(f"La columna null_values_error presenta {null_values_error} valores nulos")

La columna null_values_ocupacion presenta 46653 valores nulos
La columna null_values_vmed presenta 39608 valores nulos
La columna null_values_error presenta 88763 valores nulos


#### Comprobamos qué porcenaje del total de los datos representan los valores nulos. Para ello dividimos el total de valores nulos entre el número de total de filas.

In [53]:
total_null_values = null_values_error + null_values_ocupacion + null_values_vmed
percentage_null_values = total_null_values/data.shape[0]*100
print(f"Los valores nulos representan un {percentage_null_values:.2f}% del total")

Los valores nulos representan un 0.34% del total


#### En el peor de los casos, no hay ninguna fila que tenga más de un valor null, por lo que los valores nulos representarían aproximadamente el 0.34% del total de los registros, de manera que procedemos a su eliminación

In [59]:
len_data_with_null_values = data.shape[0]
print(f"El dataframe con valores nulos tiene {len_data_with_null_values} filas")
data_with_no_null_values = data.dropna(axis=0, how='any').copy()
len_data_with_no_null_values = data_with_no_null_values.shape[0]
print(f"El dataframe sin valores nulos tiene {len_data_with_no_null_values} filas")
removed_rows = len_data_with_null_values - len_data_with_no_null_values
print(f"Se han eliminado {removed_rows} filas")

El dataframe con valores nulos tiene 51012046 filas
El dataframe sin valores nulos tiene 50837247 filas
Se han eliminado 174799 filas


#### A partir de la columna fecha vamos a obtener otras columnas: year, month, day, name_of_day, is_weekend.
#### Esto nos permitirá posteriormente saber en qué días se ha producido mayor congestión según el año.

In [61]:
import numpy as np

data_with_no_null_values['fecha'] = pd.to_datetime(data_with_no_null_values['fecha'])

data_with_no_null_values['year'] = data_with_no_null_values['fecha'].dt.year
data_with_no_null_values['month'] = data_with_no_null_values['fecha'].dt.month
data_with_no_null_values['day'] = data_with_no_null_values['fecha'].dt.day
data_with_no_null_values['day_of_the_week'] = data_with_no_null_values['fecha'].dt.weekday
data_with_no_null_values['is_weekend'] = data_with_no_null_values['day_of_the_week'].isin([5,6]).astype(int)
data_with_no_null_values['is_holiday'] = np.where(
    (data_with_no_null_values['day'].isin([6, 8, 25]))|
    ((data_with_no_null_values['year'] == 2024) & (data_with_no_null_values['day'] == 9)),
    1.0,
    0.0
    )

data_with_no_null_values.drop(columns='fecha', inplace=True)
data_with_no_null_values

,id,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion,hora,distrito,prec,year,month,day,day_of_the_week,is_weekend,is_holiday
0,1001,M30,3660,11.0,0,59.0,N,5,00:00:00,10.0,"2,4",2021,12,1,2,0,0.0
1,1001,M30,3660,11.0,0,59.0,N,5,00:15:00,10.0,"2,4",2021,12,1,2,0,0.0
2,1001,M30,3660,11.0,0,59.0,N,5,00:30:00,10.0,"2,4",2021,12,1,2,0,0.0
3,1001,M30,3660,11.0,0,59.0,N,5,00:45:00,10.0,"2,4",2021,12,1,2,0,0.0
4,1001,M30,3660,11.0,0,59.0,N,5,01:00:00,10.0,"2,4",2021,12,1,2,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51012041,11388,M30,339,1.0,11,64.0,N,15,22:45:00,5.0,"0,0",2024,12,31,1,0,0.0
51012042,11388,M30,288,2.0,10,61.0,N,15,23:00:00,5.0,"0,0",2024,12,31,1,0,0.0
51012043,11388,M30,308,2.0,11,60.0,N,15,23:15:00,5.0,"0,0",2024,12,31,1,0,0.0
51012044,11388,M30,257,0.0,8,56.0,N,15,23:30:00,5.0,"0,0",2024,12,31,1,0,0.0


In [ ]:
cols_to_convert = ['id', 'intensidad', 'carga', 'periodo_integracion', 'year', 'month', 'day', 'day_of_the_week', 'is_weekend']
data_with_no_null_values[cols_to_convert] = data_with_no_null_values[cols_to_convert].apply(pd.to_numeric)

if data_with_no_null_values['prec'].dtype != float:
    data_with_no_null_values['prec'] = data_with_no_null_values['prec'].str.replace(',', '.').astype(float)

data_with_no_null_values.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 50837247 entries, 0 to 51012045
Data columns (total 17 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   int64  
 1   tipo_elem            object 
 2   intensidad           int64  
 3   ocupacion            float64
 4   carga                int64  
 5   vmed                 float64
 6   error                object 
 7   periodo_integracion  int64  
 8   hora                 object 
 9   distrito             float64
 10  prec                 float64
 11  year                 int32  
 12  month                int32  
 13  day                  int32  
 14  day_of_the_week      int32  
 15  is_weekend           int32  
 16  is_holiday           float64
dtypes: float64(5), int32(5), int64(4), object(3)
memory usage: 5.9+ GB


## 3. EXTRACCIÓN DE INFORMACIÓN RELEVANTE

### 3.1. COLUMNA TIPO_ELEM

#### Vamos a estudiar los datos, en primer lugar, atendiendo a la columna tipo_elem

#### De la información que se puede obtener del documento donde se explica la estructura del conjunto de datos del tráfico, sabemos que la carga no se calcula para la M30 y su valor es 0, salvo para algunos puntos de medida donde sí se realiza el cálculo de dicho parámetro. Veamos cuántos valores hay recogidos para tipo_elem == 'M30' y cuántos de ellos tienen el valor de carga == 0 y cuántos no.

In [7]:
query1 = (data_with_no_null_values['tipo_elem'] == 'M30')
m30_values = query1.sum()
print(f"Hay {m30_values} registros de tráfico recogidos para tipo_elem == 'M30'")

Hay 3541521 registros de tráfico recogidos para tipo_elem == 'M30'


#### Veamos cuántos de esos valores presentan un valor de carga == 0:

In [8]:
query2 = ((data_with_no_null_values['tipo_elem'] == 'M30') &
          (data_with_no_null_values['carga'] == 0))
m30_zero_values = query2.sum()
print(f"Hay {m30_zero_values} registros de tráfico recogidos para tipo_elem == 'M30' en los que el valor de la carga es 0")

Hay 360026 registros de tráfico recogidos para tipo_elem == 'M30' en los que el valor de la carga es 0


#### Veamos ahora cuántos de esos valores presentan un valor de carga != 0:

In [9]:
query3 = ((data_with_no_null_values['tipo_elem'] == 'M30') &
          (data_with_no_null_values['carga'] != 0))
m30_non_zero_values = query3.sum()
print(f"Hay {m30_non_zero_values} registros de tráfico recogidos para tipo_elem == 'M30' en los que el valor de la carga es distinto de 0")

Hay 3181495 registros de tráfico recogidos para tipo_elem == 'M30' en los que el valor de la carga es distinto de 0


In [10]:
percentage_m30_zero_values = m30_zero_values/m30_values*100
print(f"El {percentage_m30_zero_values:.2f}% de los registros de tráfico recogidos para tipo_elem == M30 tienen un valor de carga 0")

El 10.17% de los registros de tráfico recogidos para tipo_elem == M30 tienen un valor de carga 0


#### Por lo tanto, para la gran mayoría de los datos de tráfico recogidos por tipo_elem == 'M30', el valor de carga es distinto de 0

#### Necesitamos hacer ciertas comprobaciones respecto a algunas columnas, como por ejemplo vmed y carga ya que, como se especifica en la descripción del conjunto de datos en la página web del Ayuntamiento de Madrid, solo se recogen datos de vmed si tipo_elem es M30, por lo que tendremos muchos valores 0 para los valores de tipo_elem distintos de M30. 
#### A priori, si los valores de vmed son 0, cabe esperar que los valores de carga para dichos registros sean o muy bajos (porque no hay tráfico) o muy altos (hay mucho tráfico y por tanto congestión).
#### Comenzaremos estudiando tipo_elem == 'C30'

In [11]:
query4 = (data_with_no_null_values['tipo_elem'] == 'C30')
c30_values = query4.sum()
print(f"Hay {c30_values} registros de tráfico recogidos para tipo_elem == 'C30'")

Hay 1027534 registros de tráfico recogidos para tipo_elem == 'C30'


#### Estudiamos ahora cuántos de estos registros presentan valores de vmed == 0

In [12]:
query5 = ((data_with_no_null_values['tipo_elem'] == 'C30') &
    (data_with_no_null_values['vmed'] == 0))

c30_zero_vmed_values = query5.sum()
print(f"Hay {c30_zero_vmed_values} registros de tráfico recogidos para tipo_elem == 'C30' para los que vmed es 0")

Hay 129498 registros de tráfico recogidos para tipo_elem == 'C30' para los que vmed es 0


#### Ahora, para estos valores calculamos cuáles presentan un valor de carga == 0.

In [13]:
query6 = ((data_with_no_null_values['tipo_elem'] == 'C30') &
    (data_with_no_null_values['vmed'] == 0) &
    (data_with_no_null_values['carga'] == 0))

c30_zero_vmed_and_carga_values = query6.sum()
print(f"Hay {c30_zero_vmed_and_carga_values} registros de tráfico recogidos para tipo_elem == 'C30' para los que vmed y carga son 0")

Hay 129498 registros de tráfico recogidos para tipo_elem == 'C30' para los que vmed y carga son 0


#### Nos centramos ahora en aquellos valores de tráfico recogidos por tipo_elem == 'C30' en los que vmed no es 0. Solo tenemos que restar c30_values y c30_zero_vmed_values

In [14]:
c30_non_zero_vmed_values = c30_values - c30_zero_vmed_values
c30_non_zero_vmed_values

898036

#### Vemos cuántos de dichos valores presentan un valor de carga == 0.

In [15]:
query7 = ((data_with_no_null_values['tipo_elem'] == 'C30') &
    (data_with_no_null_values['vmed'] != 0) &
    (data_with_no_null_values['carga'] == 0))

c30_non_zero_vmed_and_zero_carga_values = query7.sum()
print(f"Hay {c30_non_zero_vmed_and_zero_carga_values} registros de tráfico recogidos para tipo_elem == 'C30' para los que vmed es 0 y carga es distinto de 0")

Hay 898036 registros de tráfico recogidos para tipo_elem == 'C30' para los que vmed es 0 y carga es distinto de 0


#### Podemos concluir que, para tipo_elem == 'C30', independientemente del valor de vmed, la carga es 0, es decir, no se recogen valores de carga para dicho tipo_elem. Se podrían eliminar todas las filas relativas a tipo_elem == 'C30'.

#### Pasamos a estudiar, por último, el tipo_elem que nos queda: 'URB'

In [16]:
query8 = (data_with_no_null_values['tipo_elem'] == 'URB')

urb_values = query8.sum()
print(f"Hay {urb_values} registros de tráfico recogidos para tipo_elem == 'URB'")

Hay 46268192 registros de tráfico recogidos para tipo_elem == 'URB'


#### Veamos cuántos de dichos valores presentan un valor de carga == 0

In [17]:
query9 = ((data_with_no_null_values['tipo_elem'] == 'URB') &
          (data_with_no_null_values['carga'] == 0))

urb_zero_carga_values = query9.sum()
print(f"Hay {urb_zero_carga_values} registros de tráfico recogidos para tipo_elem == 'URB' en los que el valor de la carga es 0")

Hay 2080692 registros de tráfico recogidos para tipo_elem == 'URB' en los que el valor de la carga es 0


In [18]:
urb_non_zero_carga_values = urb_values - urb_zero_carga_values
print(f"Hay {urb_non_zero_carga_values} registros de tráfico recogidos para tipo_elem == 'URB' en los que el valor de la carga es distinto 0")

Hay 44187500 registros de tráfico recogidos para tipo_elem == 'URB' en los que el valor de la carga es distinto 0


### 3.2. TIPADO DE COLUMNAS

#### Todas las columnas numéricas vamos a pasarlas a float.

In [ ]:
import numpy as np

cols_to_convert = ['id', 'intensidad', 'carga', 'periodo_integracion', 'year', 'month', 'day', 'day_of_the_week', 'is_weekend']
data_with_no_null_values.loc[:, cols_to_convert] = data_with_no_null_values[cols_to_convert].astype(float)

,id,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion,hora,distrito,prec,year,month,day,day_of_the_week,is_weekend
0,1001,M30,3660,11.0,0,59.0,N,5,00:00:00,10.0,2.4,2021,12,1,2,0
1,1001,M30,3660,11.0,0,59.0,N,5,00:15:00,10.0,2.4,2021,12,1,2,0
2,1001,M30,3660,11.0,0,59.0,N,5,00:30:00,10.0,2.4,2021,12,1,2,0
3,1001,M30,3660,11.0,0,59.0,N,5,00:45:00,10.0,2.4,2021,12,1,2,0
4,1001,M30,3660,11.0,0,59.0,N,5,01:00:00,10.0,2.4,2021,12,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51012041,11388,M30,339,1.0,11,64.0,N,15,22:45:00,5.0,0.0,2024,12,31,1,0
51012042,11388,M30,288,2.0,10,61.0,N,15,23:00:00,5.0,0.0,2024,12,31,1,0
51012043,11388,M30,308,2.0,11,60.0,N,15,23:15:00,5.0,0.0,2024,12,31,1,0
51012044,11388,M30,257,0.0,8,56.0,N,15,23:30:00,5.0,0.0,2024,12,31,1,0


In [46]:
if data_with_no_null_values['prec'].dtype != float:
    data_with_no_null_values['prec'] = data_with_no_null_values['prec'].str.replace(',', '.').astype(float)

data_with_no_null_values.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 50837247 entries, 0 to 51012045
Data columns (total 16 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   int64  
 1   tipo_elem            object 
 2   intensidad           int64  
 3   ocupacion            float64
 4   carga                int64  
 5   vmed                 float64
 6   error                object 
 7   periodo_integracion  int64  
 8   hora                 object 
 9   distrito             float64
 10  prec                 float64
 11  year                 int32  
 12  month                int32  
 13  day                  int32  
 14  day_of_the_week      int32  
 15  is_weekend           int32  
dtypes: float64(4), int32(5), int64(4), object(3)
memory usage: 5.5+ GB


In [50]:
data_with_no_null_values

# REVISAR POR QUÉ NO SE CAMBIA EL TIPO DE LAS COLUMNAS INT A FLOAT.

,id,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion,hora,distrito,prec,year,month,day,day_of_the_week,is_weekend
0,1001,M30,3660,11.0,0,59.0,N,5,00:00:00,10.0,2.4,2021,12,1,2,0
1,1001,M30,3660,11.0,0,59.0,N,5,00:15:00,10.0,2.4,2021,12,1,2,0
2,1001,M30,3660,11.0,0,59.0,N,5,00:30:00,10.0,2.4,2021,12,1,2,0
3,1001,M30,3660,11.0,0,59.0,N,5,00:45:00,10.0,2.4,2021,12,1,2,0
4,1001,M30,3660,11.0,0,59.0,N,5,01:00:00,10.0,2.4,2021,12,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51012041,11388,M30,339,1.0,11,64.0,N,15,22:45:00,5.0,0.0,2024,12,31,1,0
51012042,11388,M30,288,2.0,10,61.0,N,15,23:00:00,5.0,0.0,2024,12,31,1,0
51012043,11388,M30,308,2.0,11,60.0,N,15,23:15:00,5.0,0.0,2024,12,31,1,0
51012044,11388,M30,257,0.0,8,56.0,N,15,23:30:00,5.0,0.0,2024,12,31,1,0


## 4. ESCALADO DE DATOS